<a href="https://colab.research.google.com/github/JenH0620/Recommendation-System-Notebook/blob/Ch3.2-svd/Action2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install deepctr
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_feature_names

In [0]:
##DeepFM
data = pd.read_csv("ratings.csv")
sparse_features = ["userId", "movieId", 'timestamp']
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)


In [0]:
!pip install surprise
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import time

In [0]:
# 使用funkSVD
time1=time.time()

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# 使用funkSVD
algo = SVD(biased=False)

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)
time2=time.time()
print(time2-time1)

RMSE: 0.8728
RMSE: 0.8735
RMSE: 0.8726
user: 196        item: 302        r_ui = 4.00   est = 4.28   {'was_impossible': False}
164.97616720199585


In [0]:
# 使用biasSVD
time1=time.time()

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# 使用biasSVD
algo = SVD(biased=True)

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)
time2=time.time()
print(time2-time1)

RMSE: 0.8451
RMSE: 0.8451
RMSE: 0.8452
user: 196        item: 302        r_ui = 4.00   est = 4.17   {'was_impossible': False}
171.23549818992615


In [0]:
# 使用SVD++
#!pip install surprise
from surprise import SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

# 使用biasSVD
algo = SVDpp(n_epochs=20)

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)


In [0]:
##Without surprise ---- spark mllib

In [11]:
##Without surprise ---- LightFM
#!pip install lightfm
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm import cross_validation
import pandas as pd

In [21]:
##This is from the CSDN blog: https://blog.csdn.net/m0_37586991/article/details/79943400
# fetch data and format it
data = fetch_movielens(min_rating=4.0)  # only collect the movies with a rating of 4 or higher
 
# print training and testing data
print(repr(data['train']))
print(repr(data['test']))
'''repr()函数将对象转化为供解释器读取的形式'''
 
# create model
model = LightFM(loss='warp')  # warp = weighted approximate-rank pairwise
'''
warp helps us create recommendations for each user by looking at the existing user rating pairs
and predicting rankings for each, it uses the gradient descent algorithm to iteratively find the
weights that improve our prediction over time. This takes into account both the user's past rating
history content based and similar user ratings collaborative, it's a hybrid system.
WARP is an implicit feedback model: all interactions in the training matrix are treated as positive 
signals, and products that users did not interact with they implicitly do not like. The goal of the 
model is to score these implicit positives highly while assigning low scores to implicit negatives.
'''
# train model
model.fit(data['train'], epochs=30, num_threads=2)
'''
parameters: the data set we want to train it on,
            the number of epochs we want to run the training for,
            the number of threads we want to run this on
Model training is accomplished via SGD (stochastic gradient descent). This means that for every pass through 
the data — an epoch — the model learns to fit the data more and more closely. We’ll run it for 10 epochs in 
this example. We can also run it on multiple cores, so we’ll set that to 2. (The dataset in this example is 
too small for that to make a difference, but it will matter on bigger datasets.)
'''
 
def sample_recommendation(model, data, user_ids):
    # our model, our data and a list of user ids(these are users we want to generate recommendations for)
 
    # number of users and movies in training data
    n_users, n_items = data['train'].shape
 
    # generate recommendation for each user we input
    '''
    iterate through every user id that we input and say that we want the list of known positives for each line
    if M considers ratings that are 5 positive and ratings that are 4 or below negative to make the problem binary 
    much simplers
    '''
    for user_id in user_ids:
 
        # movies they already like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        '''
        data['item_labels']的类型是  <class 'numpy.ndarray'>
        data['train']的类型是  <class 'scipy.sparse.coo.coo_matrix'> 即 坐标形式的一种稀疏矩阵
            # tocsr() 的作用是  Return a copy of this matrix in Compressed Sparse Row format
            # coo_matrix.tocsr() 将把coo_matrix转化为csr_matrix，所以，
        data['train'].tocsr()的类型是 <class 'scipy.sparse.csr.csr_matrix'> 即 压缩的行稀疏矩阵
        data['train'].tocsr()[user_id] 的类型也是 <class 'scipy.sparse.csr.csr_matrix'>
        data['train'].tocsr()[user_id].indices 的类型是 <class 'numpy.ndarray'>
            # indices属性的作用是返回	CSR format index array of the matrix
            
        总之，data['train'].tocsr()[2].indices 获取  user_id=2 的观众打分为5的电影索引数组
        data['item_labels'][...]  根据索引数组，输出对应的电影名称
        
        '''
        # movies our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items))
        '''np.arange()用于创建等差数组，返回一个array对象'''
        # rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]
        '''np.argsort(x)返回数组值从小到大的索引值,np.argsort(-x)按降序排列'''
 
        # print out the results
        print("User %s" % user_id)
        print("      Known positives:")
 
        for x in known_positives[:3]:
            print("         %s" % x)
        print("      Recommended:")
        for x in top_items[:3]:
            print("         %s" % x)
 
 
sample_recommendation(model, data, [3,25,450])

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>
User 3
      Known positives:
         Seven (Se7en) (1995)
         Contact (1997)
         Starship Troopers (1997)
      Recommended:
         Scream (1996)
         Contact (1997)
         Devil's Own, The (1997)
User 25
      Known positives:
         Dead Man Walking (1995)
         Star Wars (1977)
         Fargo (1996)
      Recommended:
         English Patient, The (1996)
         Titanic (1997)
         Fargo (1996)
User 450
      Known positives:
         Contact (1997)
         George of the Jungle (1997)
         Event Horizon (1997)
      Recommended:
         Devil's Advocate, The (1997)
         Devil's Own, The (1997)
         Dante's Peak (1997)


In [0]:
##Without surprise ---- https://github.com/tushushu/imylu/blob/master/imylu/recommend/als.py
